In [113]:
import pandas as pd
import numpy as np

In [114]:
ratings_df = pd.read_csv("Dataset\last\XWines_Slim_150K_ratings.csv") 
display(ratings_df.head(10))

C:\Users\antoi\AppData\Local\Temp\ipykernel_56616\2223713249.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df = pd.read_csv("Dataset\last\XWines_Slim_150K_ratings.csv")


,RatingID,UserID,WineID,Vintage,Rating,Date
0,143,1356810,103471,1950,4.5,2021-11-02 20:52:59
1,199,1173759,111415,1951,5.0,2015-08-20 17:46:26
2,348,1164877,111395,1952,5.0,2020-11-13 05:40:26
3,374,1207665,111433,1953,5.0,2017-05-05 06:44:13
4,834,1075841,111431,1955,5.0,2016-09-14 20:18:38
5,876,1211463,111395,1955,5.0,2021-12-02 23:12:49
6,1005,1076348,111433,1955,4.5,2021-06-19 19:53:56
7,1020,1147051,111429,1955,5.0,2018-07-08 20:09:46
8,1029,1225931,111431,1955,5.0,2017-04-24 01:41:52
9,1399,1197513,111415,1958,5.0,2014-07-04 01:07:16


In [115]:
wines_df = pd.read_csv("Dataset\last\XWines_Slim_1K_wines.csv", index_col="WineID")
wines_df['WineID'] = wines_df.index
display(wines_df.head(3))

,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,WineID
WineID,,,,,,,,,,,,,,,,,
100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",100001
100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",100002
100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",100003


In [10]:

body_mapping = {
    'Very light-bodied': 1,
    'Light-bodied': 2,
    'Medium-bodied': 3,
    'Full-bodied': 4,
    'Very full-bodied': 5
}
acidity_mapping = {
    'Low': 1,
    'Medium': 2,
    'High': 3
}

type_mapping = {
    'White': 1,
    'Rosé': 2,
    'Red': 3,
    'Sparkling': 4,
    'Dessert': 5,
    'Dessert/Port': 6
}



# Apply the mappings to the DataFrame
wines_df['Body'] = wines_df['Body'].map(body_mapping)
wines_df['Acidity'] = wines_df['Acidity'].map(acidity_mapping)
wines_df['Type'] = wines_df['Type'].map(type_mapping)
display(wines_df.head(3))


,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,WineID
WineID,,,,,,,,,,,,,,,,,
100001,Espumante Moscatel,4,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,3,3,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",100001
100002,Ancellotta,3,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,3,2,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",100002
100003,Cabernet Sauvignon,3,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,4,3,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",100003


In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import scipy.sparse as sp
import numpy as np

# Features from the wine dataset
wine_features = wines_df[['Type', 'Grapes', 'Harmonize', 'ABV', 'Body', 'Acidity', 'Vintages']]

# Text Vectorization - For 'Harmonize' feature
tfidf = TfidfVectorizer(stop_words='english')
wine_tfidf = tfidf.fit_transform(wine_features['Harmonize'].fillna(''))

# Label Encoding for categorical features like 'Type', 'Body', 'Acidity', 'ABV'
label_encoder = LabelEncoder()

type_encoded = label_encoder.fit_transform(wine_features['Type'])
body_encoded = label_encoder.fit_transform(wine_features['Body'])
acidity_encoded = label_encoder.fit_transform(wine_features['Acidity'])
abv_encoded = label_encoder.fit_transform(wine_features['ABV'])

# Combine the label encoded features into a single array
wine_label_encoding = np.column_stack([type_encoded, body_encoded, acidity_encoded, abv_encoded])

# Combine the TF-IDF matrix with the label encoded features
wine_features_combined = sp.hstack([wine_tfidf, wine_label_encoding])

# Compute cosine similarity
cosine_sim = cosine_similarity(wine_features_combined, wine_features_combined)


In [117]:
#FOR WINE NAME

wines_df = wines_df.reset_index(drop=True)

def get_recommendations(wine_name, cosine_sim=cosine_sim):
    if wine_name not in wines_df['WineName'].values:
        return f"Wine '{wine_name}' not found. Please check the wine name."

    idx = wines_df[wines_df['WineName'] == wine_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:6]
    wine_indices = [i[0] for i in sim_scores]
    wine_similarities = [i[1] for i in sim_scores]
    
    recommendations = wines_df.iloc[wine_indices][['WineName', 'Type', 'Grapes', 'Body', 'Acidity', 'ABV', 'Harmonize']].copy()
    recommendations['Similarity'] = wine_similarities
    
    return recommendations



In [118]:
print(get_recommendations("Espumante Moscatel"))
#print(get_recommendations("Ancellotta"))

                        WineName       Type  \
0             Espumante Moscatel  Sparkling   
4   Do Lugar Moscatel Espumantes  Sparkling   
28                      Moscatel  Sparkling   
33  Terranova Espumante Moscatel  Sparkling   
23                      Moscatel  Sparkling   
7             Vinho Verde Branco      White   

                                            Grapes               Body Acidity  \
0                               ['Muscat/Moscato']      Medium-bodied    High   
4                        ['Muscat/Moscato Bianco']      Medium-bodied    High   
28                              ['Muscat/Moscato']      Medium-bodied    High   
33                              ['Muscat/Moscato']      Medium-bodied    High   
23                              ['Muscat/Moscato']      Medium-bodied    High   
7   ['Arinto de Bucelas', 'Loureiro', 'Trajadura']  Very light-bodied    High   

    ABV                                          Harmonize  Similarity  
0   7.5                 ['Por

In [14]:
random_user = np.random.choice(ratings_df['UserID'].unique())

user_ratings = ratings_df[ratings_df['UserID'] == random_user]
highest_rated_wine = user_ratings.loc[user_ratings['Rating'].idxmax()]

def get_wine_name(wine_id):
    wine_row = wines_df[wines_df['WineID'] == wine_id]
    if not wine_row.empty:
        return wine_row['WineName'].values[0]
    else:
        return f"Wine ID {wine_id} not found."


print(f"Random User ID: {random_user}")
print(f"Highest Rated Wine for User {random_user}:")
print(f"WineID: {highest_rated_wine['WineID']}, Rating: {highest_rated_wine['Rating']}")


wine_name = get_wine_name(highest_rated_wine['WineID'])
print(f"Wine Name: {wine_name}")
print(get_recommendations(wine_name))

Random User ID: 1196560
Highest Rated Wine for User 1196560:
WineID: 111845, Rating: 5.0
Wine Name: Côtes de Bourg
                     WineName  Type  \
244            Côtes de Bourg     3   
154     Spätburgunder Trocken     3   
249                  Bordeaux     3   
304  Francs-Côtes de Bordeaux     3   
255        Bordeaux Supérieur     3   
692    Frans Malan Cape Blend     3   

                                                Grapes  Body  Acidity   ABV  \
244             ['Merlot', 'Cabernet Franc', 'Malbec']     4        3  14.2   
154                                  ['Spätburgunder']     4        3  14.0   
249  ['Cabernet Sauvignon', 'Cabernet Franc', 'Merl...     4        3  14.5   
304                                         ['Merlot']     4        3  14.5   
255  ['Cabernet Sauvignon', 'Cabernet Franc', 'Merl...     4        3  13.8   
692       ['Pinotage', 'Cabernet Sauvignon', 'Merlot']     4        3  14.5   

                                    Harmonize  Similarity

In [119]:
user_rating_counts = ratings_df.groupby('UserID')['RatingID'].count()

most_ratings_user = user_rating_counts.idxmax()  # UserID with the most ratings
most_ratings_count = user_rating_counts.max()    # Number of ratings for that user

print(f"User {most_ratings_user} has the most ratings with {most_ratings_count} ratings.")

User 1006657 has the most ratings with 67 ratings.


In [131]:
user_ratings = ratings_df[ratings_df['UserID'] == 1006657].sort_values(by='WineID', ascending=True)
user_ratings_high = user_ratings[user_ratings['Rating'] >= 4.5]

high_rated_wines = wines_df[wines_df['WineID'].isin(user_ratings_high['WineID'])]

display(high_rated_wines[['WineName', 'WineID']])


,WineName,WineID
202,Châteauneuf-du-Pape,111457
212,Pauillac (Grand Cru Classé),111495
215,Grand Vintage Brut Champagne,111521
230,Emilien,111687
232,Corton-Charlemagne Grand Cru,111713
277,Pomerol,112511
316,Listrac-Médoc,113193
523,Priorat,155438


In [139]:
wine_name = 'Châteauneuf-du-Pape'

idx = wines_df[wines_df['WineName'] == wine_name].index[0]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

wine_indices = [i[0] for i in sim_scores]
wine_similarities = [i[1] for i in sim_scores]

recommendations = wines_df.iloc[wine_indices][['WineName', 'Type', 'Grapes', 'Body', 'Acidity', 'ABV', 'Harmonize']].copy()
recommendations['Similarity'] = wine_similarities



In [141]:
wine_to_find = 'Listrac-Médoc'
if wine_to_find in recommendations['WineName'].values:
    
    wine_row = recommendations[recommendations['WineName'] == wine_to_find]
    
    similarity_score = wine_row['Similarity'].values[0]
    
    print(f"The similarity score for {wine_to_find} is: {similarity_score}")
else:
    print(f"{wine_to_find} is not in the recommendations list.")

The similarity score for Listrac-Médoc is: 0.9963638377827774
